In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
'''
3) Real-life dataset
'''

df = pd.read_csv('./Datasets/communities.csv', header=None)
df[0] = 1
df.drop([i for i in range(1, 5)], axis=1, inplace=True) # These columns are not predictive according to the dataset
df.columns = [i for i in range(df.shape[1])] # Rename columns
df = df.replace('?', np.NaN).astype(np.float64)

print(df.head())

   0     1     2     3     4     5     6     7     8     9    ...    114  \
0  1.0  0.19  0.33  0.02  0.90  0.12  0.17  0.34  0.47  0.29  ...   0.12   
1  1.0  0.00  0.16  0.12  0.74  0.45  0.07  0.26  0.59  0.35  ...   0.02   
2  1.0  0.00  0.42  0.49  0.56  0.17  0.04  0.39  0.47  0.28  ...   0.01   
3  1.0  0.04  0.77  1.00  0.08  0.12  0.10  0.51  0.50  0.34  ...   0.02   
4  1.0  0.01  0.55  0.02  0.95  0.09  0.05  0.38  0.38  0.23  ...   0.04   

    115   116   117   118  119  120   121   122   123  
0  0.26  0.20  0.06  0.04  0.9  0.5  0.32  0.14  0.20  
1  0.12  0.45   NaN   NaN  NaN  NaN  0.00   NaN  0.67  
2  0.21  0.02   NaN   NaN  NaN  NaN  0.00   NaN  0.43  
3  0.39  0.28   NaN   NaN  NaN  NaN  0.00   NaN  0.12  
4  0.09  0.02   NaN   NaN  NaN  NaN  0.00   NaN  0.03  

[5 rows x 124 columns]


In [3]:
'''
Part 1) Fill in missing values 
'''
df.fillna(df.mean(), inplace=True)
# print(df.columns[df.isnull().any()].tolist())

N_examples = df.shape[0]
M_cols = df.shape[1]

data = np.array(df, dtype=np.float64)

print(df.head())

   0     1     2     3     4     5     6     7     8     9    ...    114  \
0  1.0  0.19  0.33  0.02  0.90  0.12  0.17  0.34  0.47  0.29  ...   0.12   
1  1.0  0.00  0.16  0.12  0.74  0.45  0.07  0.26  0.59  0.35  ...   0.02   
2  1.0  0.00  0.42  0.49  0.56  0.17  0.04  0.39  0.47  0.28  ...   0.01   
3  1.0  0.04  0.77  1.00  0.08  0.12  0.10  0.51  0.50  0.34  ...   0.02   
4  1.0  0.01  0.55  0.02  0.95  0.09  0.05  0.38  0.38  0.23  ...   0.04   

    115   116       117       118       119       120   121       122   123  
0  0.26  0.20  0.060000  0.040000  0.900000  0.500000  0.32  0.140000  0.20  
1  0.12  0.45  0.163103  0.076708  0.698589  0.440439  0.00  0.195078  0.67  
2  0.21  0.02  0.163103  0.076708  0.698589  0.440439  0.00  0.195078  0.43  
3  0.39  0.28  0.163103  0.076708  0.698589  0.440439  0.00  0.195078  0.12  
4  0.09  0.02  0.163103  0.076708  0.698589  0.440439  0.00  0.195078  0.03  

[5 rows x 124 columns]


In [5]:
'''
Part 2) Fit data and report 5-fold cross-validation error
'''
# Normal equation with regularization (lambda_reg=0 means no regularization)
def fit(X, y, lambda_reg=0):
    identity = np.identity(X.shape[1])
    # identity[0, 0] = 0 # We do not penalize the bias term

    X_square = np.matmul(np.transpose(X), X) + lambda_reg * identity
    X_square_inverse = np.linalg.pinv(X_square)
    weights = np.matmul(np.matmul(X_square_inverse, np.transpose(X)), y)

    return weights

# Gradient descent
def gradient_descent(X, y, lambda_reg=0, alpha=1e-2, epochs=5000, weights=None):
    '''
    Implementation of vectorized gradient descent with L2 regularization support
    :param X: The input matrix N x M, where each row is an example
    :param y: The output, N x 1
    :param lambda_reg: Regularization hyperparamter (0 means no regularization)
    :param alpha: Learning rate
    :param epochs: Number of cycles over training data
    :param weights: Initial weights can be supplied if desired
    :return: The optimal weights after gradient descent
    '''
    weights = weights if weights is not None else np.random.uniform(high=10, size=[M_cols - 1])
    N = len(X)
    for epoch in range(epochs):
        weights = weights - alpha / N * ( np.matmul(np.transpose(X), np.matmul(X, weights) - y) + lambda_reg * weights)

    return weights

def mean_square_error(X, y, W):
    y_hat = np.matmul(X, W)
    mean_square_err = np.sum(np.square(y - y_hat)) / len(y)

    return mean_square_err


def cross_validation_split(X, n_folds=5, filename="file", write_to_csv=False):
    N = len(X) // 5
    pairs = []
    for i in range(n_folds):
        fold_train1 = X[0:i * N]
        if i < n_folds - 1:
            fold_test = X[i*N:(i+1)*N]
            fold_train2 = X[(i+1)*N:]
        else:
            fold_test = X[i*N:]
            fold_train2 = X[N:N]

        df_train = pd.DataFrame(np.concatenate((fold_train1, fold_train2)))
        df_test = pd.DataFrame(fold_test)

        if write_to_csv:
            df_train.to_csv('./Datasets/' + filename + '-train' + str(i + 1) + '.csv', header=False, index=False)
            df_test.to_csv('./Datasets/' + filename + '-test' + str(i + 1) + '.csv', header=False, index=False)

        pairs.append({'train': df_train, 'test': df_test})

    return pairs


In [6]:
# Generate the files for 5-fold cross-validation
# splits = [{'train':<train_data>, 'test': <test_data>}, {'train':<train_data>, 'test': <test_data>}, {'train':<train_data>, 'test': <test_data>}\
# {'train':<train_data>, 'test': <test_data>}, {'train':<train_data>, 'test': <test_data>}]
splits = cross_validation_split(data, 5, 'CandC', False)

In [7]:
MSEs_closed_form = []
all_weights = []
for pair in splits:
    X_train = pair['train'].drop([M_cols - 1], axis=1)
    y_train = pair['train'][M_cols-1]
    X_test = pair['test'].drop([M_cols - 1], axis=1)
    y_test = pair['test'][M_cols - 1]

    weights = fit(X_train, y_train)
    MSEs_closed_form.append(mean_square_error(X_test, y_test, weights))
    all_weights.append(weights)
    

In [8]:
MSEs_gd = []
all_weights_gd = []
weights = np.random.uniform(high=10., size=[M_cols - 1])
for pair in splits:
    X_train = pair['train'].drop([M_cols - 1], axis=1)
    y_train = pair['train'][M_cols - 1]
    X_test = pair['test'].drop([M_cols - 1], axis=1)
    y_test = pair['test'][M_cols - 1]

    weights_gd = gradient_descent(np.array(X_train), np.array(y_train), epochs=20000, weights=np.copy(weights))
    MSEs_gd.append(mean_square_error(X_test, y_test, weights_gd))
    all_weights_gd.append(weights_gd)


In [9]:
print('least squares:', np.average(MSEs_closed_form))
print('gradient descent:', np.average(MSEs_gd))


least squares: 0.18373121756680147
gradient descent: 0.5513726961887607
